In [ ]:
!pip install transformers datasets torch accelerate

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    GPT2LMHeadModel,
    GPT2Config,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)

# 1. Cargar el Dataset y verificar
dataset_file = "/content/dataset_TRAVIAN_CLEAN_V2.txt"

# Verificación rápida
with open(dataset_file, "r", encoding="utf-8") as f:
    print("Primeras líneas del dataset:", f.readlines()[:5])

dataset = load_dataset("text", data_files=dataset_file)
dataset = dataset["train"].train_test_split(test_size=0.1)

# 2. Tokenizador (Byte-level es CLAVE para ASCII)
model_name = "openai-community/gpt2" # GPT-2 Small
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 3. Preprocesamiento (Aumentado a 1024 tokens para dibujos grandes)
def preprocess(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024, # AUMENTADO: El ASCII gasta muchos tokens
        padding="max_length"
    )

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["text"])

# 4. Configurar el Modelo (Sin cuantización 4-bit, usaremos fp16 nativo)
# GPT-2 Small cabe en la memoria de Colab sin trucos raros.
model = GPT2LMHeadModel.from_pretrained(model_name)

# 5. Configuración de Entrenamiento
training_args = TrainingArguments(
    output_dir="./ascii_travian_gpt2",
    overwrite_output_dir=True,
    num_train_epochs=5,           # Empezamos con 5, si queda flojo subimos a 10
    per_device_train_batch_size=4, # Bajamos batch size para compensar el max_length de 1024
    gradient_accumulation_steps=4, # Acumulamos gradientes para estabilidad
    learning_rate=5e-5,           # Un pelín más alto para que aprenda rápido
    weight_decay=0.01,
    warmup_steps=100,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=True,                    # Usar precisión media (rápido y ligero)
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

# 6. ¡ENTRENAR!
print("Iniciando entrenamiento...")
trainer.train()

# 7. Guardar el modelo final
trainer.save_model("./modelo_final_ascii")
tokenizer.save_pretrained("./modelo_final_ascii")
print("Modelo guardado.")

Primeras líneas del dataset: ['<|startoftext|>\n', 'User: Dibuja un ASCII art de animals aardvarks\n', 'AI:\n', '       _.---._    /\\\\                             \n', '    ./\'       "--`\\//                             \n']


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

Iniciando entrenamiento...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Re-configuramos SOLO el entrenamiento para no recargar los datos
from transformers import TrainingArguments, Trainer

# 1. Nueva configuración (Más rápida: 1 época)
training_args = TrainingArguments(
    output_dir="./ascii_travian_gpt2",
    overwrite_output_dir=True,
    num_train_epochs=1,            # CAMBIO: Solo 1 vuelta. Con 100k datos es suficiente.
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=100,
    logging_steps=50,              # Que nos avise cada 50 pasos
    save_steps=500,                # Guardar checkpoint cada 500 pasos
    save_total_limit=2,            # Solo guardar los 2 últimos checkpoints para no llenar el disco
    eval_strategy="no",            # Saltamos evaluación para ir rápido
    save_strategy="steps",
    fp16=True,
    report_to="none"
)

# 2. Actualizamos el "Entrenador"
trainer = Trainer(
    model=model,                   # Usa el modelo que ya está en memoria
    args=training_args,
    train_dataset=tokenized_dataset["train"], # Usa los datos que ya están en memoria
    data_collator=data_collator,
)

# 3. ¡A ENTRENAR DE NUEVO!
print("Re-iniciando entrenamiento (Versión rápida)...")
trainer.train()

# 4. Guardar al final
trainer.save_model("./modelo_final_ascii")
tokenizer.save_pretrained("./modelo_final_ascii")
print("¡Entrenamiento terminado y guardado!")

Re-iniciando entrenamiento (Versión rápida)...


Step,Training Loss
50,1.356600
100,1.294500
150,1.262900
200,1.289900
250,1.200300
300,1.295700
350,1.199300
400,1.276400
450,1.272200
500,1.258900


¡Entrenamiento terminado y guardado!


In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# 1. Cargar modelo (asegúrate de que la ruta sea correcta)
ruta_modelo = "./modelo_final_ascii"
tokenizer = AutoTokenizer.from_pretrained(ruta_modelo)
model = AutoModelForCausalLM.from_pretrained(ruta_modelo)

# 2. Configurar el generador
# Usamos un truco: 'max_length' es el total (prompt + generación).
# Lo ponemos en 1024 que es el límite de GPT-2.
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

prompts = [
    "<|startoftext|>\nUser: Dibuja un ASCII art de un mapa\nAI:\n",
    "<|startoftext|>\nUser: Dibuja un ASCII art de un edificio romano\nAI:\n",
    "<|startoftext|>\nUser: Dibuja un ASCII art de animals dragons\nAI:\n"
]

print("🎨 Generando Arte ASCII (Intento 2 - Sin romper la GPU)...\n")

for prompt in prompts:
    print(f"--- Generando para: {prompt.strip()} ---")

    output = generator(
        prompt,
        max_length=1024,    # EL LÍMITE FÍSICO DE GPT-2
        temperature=0.6,    # Creatividad baja para mantener líneas rectas
        top_p=0.9,
        repetition_penalty=1.2, # Subí esto un poco para que no repita bucles
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        truncation=True
    )

    # Extraemos solo el dibujo, quitando el prompt para que se vea limpio
    generated_art = output[0]["generated_text"]

    # Opcional: Limpiar el prompt para ver solo el dibujo
    clean_art = generated_art.replace(prompt, "")

    print(clean_art)
    print("\n" + "="*50 + "\n")

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🎨 Generando Arte ASCII (Intento 2 - Sin romper la GPU)...

--- Generando para: <|startoftext|>
User: Dibuja un ASCII art de un mapa
AI: ---


Both `max_new_tokens` (=256) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


(  )DIAA |><C] <J>) J)R[][][ ] [ ][ ][ ][ //]] </>>>>>><<<</>>>>>>>>>>>>><<<<<>. \/ /.\|`--.__|.'|||||| ||    (___,----'==='=======')-'==="===" == '====="" === = >-~ -X@*+XX#==+=###==}_//|  *\\|\  LGBV&LG/#!|;##%|;#####%%|  @@@;;;%%|;#######|;####|;|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;########|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|;####|


--- Generando para: <|startoftext|>
User: Dibuja un ASCII art de un edificio romano
AI: ---


Both `max_new_tokens` (=256) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


(O)NOMMIMMOXNYMMMB  |--------------------------.--------'----------''-----------''.------------------------------`-------------'||.'-----|_.-'.------..__.-;---,`.-------.,/~\  / \ `-,',-.~~! ~\ ,-(,'.) |||||||//|  )|||  <|startandall></><{*}>.  {*/@}} </><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><


--- Generando para: <|startoftext|>
User: Dibuja un ASCII art de animals dragons
AI: ---
(+) crown princess  <><'='="`""=,_,'=""==',-""=,-.__<>""""'"`"""'`"-._  |  `-./\|/___|`._____.'| ||||||||  /  \|/\/|  _|||| .-.____.-~--~~---_.  )||  ( __|.)  | '-'|  '.-|  :|  ,'/| | |     [ ] >_>  ()>()>   (<>)   *>,    -')>  <>        ~>-())>((|)>>()|)>.|  //</>//>\\>\\>@>\>|         \\|/<<|\\>*/>|     +'>|/<>@@@>\>|  ;|/

In [ ]:
import textwrap

# Configuración corregida para evitar el error de tokens y la repetición
generation_kwargs = {
    "max_new_tokens": 800,      # Forzamos 800 pasos nuevos (ignora max_length)
    "temperature": 0.75,        # Subimos un poco para que sea más creativo y no se atasque
    "top_p": 0.92,
    "repetition_penalty": 1.3,  # ¡IMPORTANTE! Subimos esto para matar los bucles <><><>
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id
}

def imprimir_bien(texto_ascii):
    """Función auxiliar para ver el ASCII correctamente en Colab"""
    print("\n" + "⬇️" * 20)
    lines = texto_ascii.split('\n')
    for line in lines:
        # Imprimimos la línea tal cual, sin dejar que Colab la recorte
        print(line)
    print("⬆️" * 20 + "\n")

print("🎨 Generando Arte ASCII (Intento 3 - Parámetros corregidos)...\n")

prompts = [
    "<|startoftext|>\nUser: Dibuja un ASCII art de animals dragons\nAI:\n",
    "<|startoftext|>\nUser: Dibuja un ASCII art de un mapa\nAI:\n"
]

for prompt in prompts:
    print(f"--- Intentando dibujar: {prompt.strip()} ---")

    # Generamos
    output = generator(prompt, **generation_kwargs)
    generated_text = output[0]["generated_text"]

    # Limpiamos el prompt para ver solo el arte
    arte_limpio = generated_text.replace(prompt, "")

    # Imprimimos con la función especial
    imprimir_bien(arte_limpio)

🎨 Generando Arte ASCII (Intento 3 - Parámetros corregidos)...

--- Intentando dibujar: <|startoftext|>
User: Dibuja un ASCII art de animals dragons
AI: ---

⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️
(  ) cows-boobies cats unicorns?s@adventures.edu/>||</><\< |'`-'_.'}/ \ /.\-/_._\ <--~) '-. `(_.-') _.--'' || __,^-|`.__|-----..___...---~~=------==+---------+-------+.=======+=+++++++++|----#=-'.:::::::' ::::::::::\  ;<<>>*###<<;|>##*//#####|#####|#|######|#  #|  ##|   ### |######################## ######[]#####.0|#|###    @@@@@@@;;@@@@@@@@;;;;;,;;;;;;;;;;;;;;;;;;;;,%%%%%%'  //','  '.\ '.;'\ ',',.,;;;;',%;,% ,;%%'',;%%',;'%,  :::  ;; ;|,;;;;;;;;;;;;'/.'%.;;';|...':,. ::   :,      (()), ;;;,,.:...>>>>>>>>>>>>>>'>>  !;''''```,,!!!!!!!!!!!;>,!!''!''!' <!!!!!!!!!> !!;'    %.%%%%%%        ```````<!!!;,;;<$$$$$$$P"",$$F,c$hccnk??u???$,`$$ $$$$$$$,?$$$N.""""?$$$C,CCc$$$$$$$?dMMMML>".`4$$$$$$$?$$$n.?$$$$$h C$?$c    ""?????" "T$$$$$$$$$$$"    .,  '' .,,,,xMMM"      MMMM       $$L,`"""???>?$$$$$$nJ  

In [ ]:
# Re-configuración para ENTRENAMIENTO PESADO
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./ascii_travian_gpt2_heavy", # Cambié el nombre para no mezclar
    overwrite_output_dir=True,
    num_train_epochs=15,           # CAMBIO: De 1 a 15. Vamos a forzar el aprendizaje.
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,              # Más calentamiento
    logging_steps=100,
    save_steps=2000,               # Guardar menos a menudo para no llenar el disco
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,                   # Usamos el modelo que ya tiene 1 época (ya sabe algo)
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
)

print("🚀 Iniciando Entrenamiento Pesado (15 Épocas)...")
print("Ve a por comida, esto va a tomar un rato.")
trainer.train()

# Guardamos el modelo definitivo
trainer.save_model("./modelo_heavy_ascii")
tokenizer.save_pretrained("./modelo_heavy_ascii")
print("¡Modelo Pesado Guardado!")

NameError: name 'tokenized_dataset' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import os

# Nombre de la carpeta en tu Drive
destino = "/content/drive/My Drive/Backup_Modelo_Travian_V1"

# Copiamos la carpeta del modelo
if os.path.exists("./modelo_final_ascii"):
    shutil.copytree("./modelo_final_ascii", destino)
    print(f"✅ ¡Guardado con éxito! Revisa en tu Google Drive la carpeta: {destino}")
else:
    print("❌ No encuentro la carpeta './modelo_final_ascii'. ¿Seguro que se guardó bien antes?")

✅ ¡Guardado con éxito! Revisa en tu Google Drive la carpeta: /content/drive/My Drive/Backup_Modelo_Travian_V1


In [ ]:
shutil.copy("dataset_TRAVIAN_CLEAN_V2.txt", "/content/drive/My Drive/dataset_TRAVIAN_CLEAN_V2.txt")
print("✅ Dataset guardado en Drive.")

✅ Dataset guardado en Drive.
